<a href="https://colab.research.google.com/github/hselino/complaint_analysis/blob/main/class_multilingual_no_seg_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets scikit-learn evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and

In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/translated1000.csv")
df = df.drop(columns=["Unnamed: 0"], errors='ignore')

In [ ]:
df.head()

,şikayet,issue
0,Kendim ve homepoint finansal olarak kısmen ned...,Ödeme süreci sırasında sorun
1,Equifax ile kredi raporumda bir sahtekarlık uy...,Raporunuzun uygunsuz kullanımı
2,"Son zamanlarda, bir aldatmaca koşan ve para ve...",Borçlu Borç Alma Denemeleri
3,Dağ Zirvesi Financial'tan bir kredi için ön pl...,Borçlu veya Servic ile iletişime geçilemiyor
4,Arizona Doğum Tarihi SS Transunion LLC Tüketic...,Raporunuzdaki yanlış bilgiler


In [ ]:
df = df.dropna()
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['issue'])

# Save mapping for later
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
num_labels = len(label_mapping)

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['şikayet'].tolist(), df['encoded_label'].tolist(), test_size=0.2, random_state=42
)

In [ ]:
from transformers import AutoTokenizer

model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

In [ ]:
import torch

class TurkishDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = TurkishDataset(train_encodings, train_labels)
val_dataset = TurkishDataset(val_encodings, val_labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=num_labels)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir='./logs',
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ayseulasli2727 (ayseulasli2727-istanbul-k-lt-r-niversitesi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,3.239083,0.280000,0.161879,0.119646,0.280000
2,No log,3.008940,0.315000,0.211845,0.187162,0.315000
3,No log,2.923662,0.305000,0.223878,0.258387,0.305000
4,No log,2.893144,0.310000,0.218431,0.262930,0.310000
5,3.085700,2.922705,0.300000,0.222222,0.187851,0.300000
6,3.085700,2.905961,0.340000,0.259637,0.256121,0.340000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

TrainOutput(global_step=600, training_loss=2.9837103780110676, metrics={'train_runtime': 308.8639, 'train_samples_per_second': 25.901, 'train_steps_per_second': 3.238, 'total_flos': 315971393126400.0, 'train_loss': 2.9837103780110676, 'epoch': 6.0})

In [ ]:
model_path = "/content/drive/MyDrive/fine_tuned_model-miltilingual-no-seg"

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/fine_tuned_model-miltilingual-no-seg/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_model-miltilingual-no-seg/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_model-miltilingual-no-seg/vocab.txt',
 '/content/drive/MyDrive/fine_tuned_model-miltilingual-no-seg/added_tokens.json',
 '/content/drive/MyDrive/fine_tuned_model-miltilingual-no-seg/tokenizer.json')

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
def predict(texts):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predictions = torch.argmax(probs, dim=-1)

    predictions = predictions.cpu().numpy()
    labels = label_encoder.inverse_transform(predictions)
    return labels

In [ ]:
sample_texts = [
    "15 Mart 2025 tarihinde Adana Barajyolu Ziraat Bankası şube ATM'sinde 13 bin para yatırma işlemi gerçekleştirdim, ancak ATM para haznesine nakiti koyduğumda para haznesi kapandı ve kartımı geri iade etti, (normalde para sayıp ardından ekrana tutarın yansıması onayla kısmı gelmesi gerekti) yani paramı yuttu. Ortak ATM olduğu için işlemi QNB kartım ile gerçekleştirdim, Ziraat Bankası aradığımda kendi bankanı ara dedi. QNB bankası ise talep oluşturduklarını söyledi. Tam 15 gün geçti sonuçlanmadı, dönüş yapılmıyor dendi. Bu sefer Ziraat Barajyolu şubesine gittim tekrar, sert ve kırıcı üsluplarıyla karşılaştım. Bana kendi bankanın şube müdürlüğüne ulaşması lazım! Yazılı dilekçe ver o bankadan! Dedi. Zaman ayırıp tekrar gittim böyle bir işlem olmadığını işleyişinde bu şekilde olmadığını söylediler. Bugün 23. Gün alenen parama el konulduğunu düşünüyorum. Bu kadar işlek bir yerde her gün para sayılır fazla tutar bellidir. Bir talebe bu kadar süre dönülmemesi ve iş bilmemeleri şok edici. Gelişme olursa yazımı güncelleyeceğim"
]

predict(sample_texts)

array(['Bir Hesabı Yönetmek'], dtype=object)